# PostScript


In [ ]:
var fso = new ActiveXObject("Scripting.FileSystemObject");

// 打開PostScript檔案
function openPSFile(filePath) {
    if (!fso.FileExists(filePath)) {
        WScript.Echo("File not found: " + filePath);
        return null;
    }
    var file = fso.OpenTextFile(filePath, 1);
    return file;
}

// 讀取PostScript檔案的所有內容
function readPSFile(file) {
    var content = "";
    while (!file.AtEndOfStream) {
        content += file.ReadLine() + "\n";
    }
    return content;
}

// 抓取特定頁面標籤區
function getPageSections(content) {
    var pages = content.split("%%Page: ");
    var pageSections = [];
    for (var i = 1; i < pages.length; i++) {
        pageSections.push("%%Page: " + pages[i]);
    }
    return pageSections;
}

// 創建指令物件
function createCommand(commandLines) {
    return commandLines.join("\n");
}

// 將指令物件推送到特定頁面標籤區
function pushCommandToPageSection(pageSection, command) {
    var setupIndex = pageSection.indexOf("%%BeginPageSetup");
    if (setupIndex === -1) {
        WScript.Echo("Page setup section not found");
        return pageSection;
    }
    var endSetupIndex = pageSection.indexOf("%%EndPageSetup", setupIndex);
    if (endSetupIndex === -1) {
        WScript.Echo("Page setup end section not found");
        return pageSection;
    }
    var beforeSetup = pageSection.substring(0, setupIndex + 16); // "%%BeginPageSetup" length is 16
    var afterSetup = pageSection.substring(endSetupIndex);
    return beforeSetup + "\n" + command + "\n" + afterSetup;
}

// 保存並關閉檔案
function saveAndCloseFile(filePath, content) {
    var file = fso.OpenTextFile(filePath, 2, true);
    file.Write(content);
    file.Close();
}

// 主函數
function PSPageCommand(filePath, pageIndex, commandLines) {
    var file = openPSFile(filePath);
    if (file === null) return;

    var content = readPSFile(file);
    file.Close();

    var pageSections = getPageSections(content);

    if (pageIndex < 1 || pageIndex > pageSections.length) {
        WScript.Echo("Page index out of range");
        return;
    }

    var command = createCommand(commandLines);
    pageSections[pageIndex - 1] = pushCommandToPageSection(pageSections[pageIndex - 1], command);

    var newContent = pageSections.join("\n");
    saveAndCloseFile(filePath, newContent);
}

// 使用範例
var filePath = "path\\to\\your\\file.ps";
var pageIndex = 1; // 指定頁面索引
var commandLines = ["<< /Staple 3 >> setpagedevice"];

PSPageCommand(filePath, pageIndex, commandLines);


# V2

In [ ]:
var fso = new ActiveXObject("Scripting.FileSystemObject");

// Open the PostScript file
function openPSFile(filePath) {
    if (!fso.FileExists(filePath)) {
        WScript.Echo("File not found: " + filePath);
        return null;
    }
    var file = fso.OpenTextFile(filePath, 1);
    return file;
}

// Read the entire contents of the PostScript file
function readPSFile(file) {
    var content = "";
    while (!file.AtEndOfStream) {
        content += file.ReadLine() + "\n";
    }
    return content;
}

// Retrieve specific page sections
function getPageSections(content) {
    var pages = content.split(/(%%Page:\s*\(\d+\)\s*\d+)/);
    var pageSections = [];
    for (var i = 1; i < pages.length; i += 2) {
        pageSections.push(pages[i] + pages[i + 1]);
    }
    return pageSections;
}

// Create command object
function createCommand(commandLines) {
    return commandLines.join("\n");
}

// Push command object to specific page section
function pushCommandToPageSection(pageSection, command, isEndCommand) {
    if (isEndCommand) {
        return pageSection + "\n" + command + "\n";
    } else {
        var setupIndex = pageSection.indexOf("%%BeginPageSetup");
        if (setupIndex === -1) {
            WScript.Echo("Page setup section not found");
            return pageSection;
        }
        var endSetupIndex = pageSection.indexOf("%%EndPageSetup", setupIndex);
        if (endSetupIndex === -1) {
            WScript.Echo("Page setup end section not found");
            return pageSection;
        }
        var beforeSetup = pageSection.substring(0, setupIndex + 16); // "%%BeginPageSetup" length is 16
        var afterSetup = pageSection.substring(endSetupIndex);
        return beforeSetup + "\n" + command + "\n" + afterSetup;
    }
}

// Save and close file
function saveAndCloseFile(filePath, content) {
    var file = fso.OpenTextFile(filePath, 2, true);
    file.Write(content);
    file.Close();
}

// Main function
function PostScriptMain(filePath, startPageIndex, endPageIndex, startCommandLines, endCommandLines) {
    var file = openPSFile(filePath);
    if (file === null) return;

    var content = readPSFile(file);
    file.Close();

    var pageSections = getPageSections(content);

    if (startPageIndex < 1 || startPageIndex > pageSections.length ||
        endPageIndex < 1 || endPageIndex > pageSections.length ||
        startPageIndex > endPageIndex) {
        WScript.Echo("Page index out of range or invalid range");
        return;
    }

    var startCommand = createCommand(startCommandLines);
    var endCommand = createCommand(endCommandLines);

    pageSections[startPageIndex - 1] = pushCommandToPageSection(pageSections[startPageIndex - 1], startCommand, false);
    pageSections[endPageIndex - 1] = pushCommandToPageSection(pageSections[endPageIndex - 1], endCommand, true);

    var newContent = pageSections.join("");
    saveAndCloseFile(filePath, newContent);
}

// Example usage
var filePath = "path\\to\\your\\file.ps";
var startPageIndex = 1; // Start binding from this page index
var endPageIndex = 3; // End binding at this page index
var startCommandLines = [
    "%%BeginFeature: *FFStaple UpperLeftSinglePort",
    "3 dict begin",
    "    /Staple 3 def",
    "    /OutputType (FINISHER TRAY) def",
    "    2 dict begin",
    "    /Type 7 def",
    "    /Location 0 def currentdict end",
    "    /StapleDetails exch def currentdict end",
    "setpagedevice",
    "%%EndFeature",
    "} stopped cleartomark"
];

var endCommandLines = [
    "%%BeginFeature: *FFStaple None",
    "1 dict dup/Staple 0 put setpagedevice",
    "%%EndFeature",
    "} stopped cleartomark"
];

main(filePath, startPageIndex, endPageIndex, startCommandLines, endCommandLines);


# V3

In [ ]:
var fso = new ActiveXObject("Scripting.FileSystemObject");

// Open the PostScript file
function openPSFile(filePath) {
    if (!fso.FileExists(filePath)) {
        WScript.Echo("File not found: " + filePath);
        return null;
    }
    var file = fso.OpenTextFile(filePath, 1);
    return file;
}

// Read the entire contents of the PostScript file
function readPSFile(file) {
    var content = "";
    while (!file.AtEndOfStream) {
        content += file.ReadLine() + "\n";
    }
    return content;
}

// Retrieve specific page sections
function getPageSections(content) {
    var pages = content.split(/(%%Page:\s*\(\d+\)\s*\d+)/);
    var pageSections = [];
    for (var i = 1; i < pages.length; i += 2) {
        pageSections.push(pages[i] + pages[i + 1]);
    }
    return pageSections;
}

// Create command object
function createCommand(commandLines) {
    return commandLines.join("\n");
}

// Insert command object into a specific page section
function insertCommandToPageSection(pageSection, command, isEndCommand) {
    if (isEndCommand) {
        return pageSection + "\n" + command + "\n";
    } else {
        var setupIndex = pageSection.indexOf("%%BeginPageSetup");
        if (setupIndex === -1) {
            WScript.Echo("Page setup section not found");
            return pageSection;
        }
        var endSetupIndex = pageSection.indexOf("%%EndPageSetup", setupIndex);
        if (endSetupIndex === -1) {
            WScript.Echo("Page setup end section not found");
            return pageSection;
        }
        var beforeSetup = pageSection.substring(0, endSetupIndex + 14); // "%%EndPageSetup" length is 14
        var afterSetup = pageSection.substring(endSetupIndex + 14);
        return beforeSetup + "\n" + command + "\n" + afterSetup;
    }
}

// Save and close file
function saveAndCloseFile(filePath, content) {
    var file = fso.OpenTextFile(filePath, 2, true);
    file.Write(content);
    file.Close();
}

// Main function
function main(filePath, startPageIndex, endPageIndex, startCommandLines, endCommandLines) {
    var file = openPSFile(filePath);
    if (file === null) return;

    var content = readPSFile(file);
    file.Close();

    var pageSections = getPageSections(content);

    if (startPageIndex < 1 || startPageIndex > pageSections.length ||
        endPageIndex < 1 || endPageIndex > pageSections.length ||
        startPageIndex > endPageIndex) {
        WScript.Echo("Page index out of range or invalid range");
        return;
    }

    var startCommand = createCommand(startCommandLines);
    var endCommand = createCommand(endCommandLines);

    pageSections[startPageIndex - 1] = insertCommandToPageSection(pageSections[startPageIndex - 1], startCommand, false);
    pageSections[endPageIndex - 1] = insertCommandToPageSection(pageSections[endPageIndex - 1], endCommand, true);

    var newContent = pageSections.join("");
    saveAndCloseFile(filePath, newContent);
}

// Example usage
var filePath = "path\\to\\your\\file.ps";
var startPageIndex = 1; // Start binding from this page index
var endPageIndex = 3; // End binding at this page index
var startCommandLines = [
    "%%BeginFeature: *FFStaple UpperLeftSinglePort",
    "3 dict begin",
    "    /Staple 3 def",
    "    /OutputType (FINISHER TRAY) def",
    "    2 dict begin",
    "    /Type 7 def",
    "    /Location 0 def currentdict end",
    "    /StapleDetails exch def currentdict end",
    "setpagedevice",
    "%%EndFeature",
    "} stopped cleartomark"
];

var endCommandLines = [
    "%%BeginFeature: *FFStaple None",
    "1 dict dup/Staple 0 put setpagedevice",
    "%%EndFeature",
    "} stopped cleartomark"
];

main(filePath, startPageIndex, endPageIndex, startCommandLines, endCommandLines);


# V3

In [ ]:
var fso = new ActiveXObject("Scripting.FileSystemObject");

// Open the PostScript file
function openPSFile(filePath) {
    if (!fso.FileExists(filePath)) {
        WScript.Echo("File not found: " + filePath);
        return null;
    }
    var file = fso.OpenTextFile(filePath, 1);
    return file;
}

// Read the entire contents of the PostScript file
function readPSFile(file) {
    var content = "";
    while (!file.AtEndOfStream) {
        content += file.ReadLine() + "\n";
    }
    return content;
}

// Save and close file
function saveAndCloseFile(filePath, content) {
    var file = fso.OpenTextFile(filePath, 2, true);
    file.Write(content);
    file.Close();
}

// Insert start command after %%BeginPageSetup and end command before %%Page:
function insertCommands(content, startPageIndex, endPageIndex, startCommand, endCommand) {
    var lines = content.split("\n");
    var pageCounter = 0;
    var newContent = [];
    var insertStartCommand = false;
    var insertEndCommand = false;

    for (var i = 0; i < lines.length; i++) {
        var line = lines[i];
        
        if (line.match(/%%Page:\s*\(\d+\)\s*\d+/)) {
            pageCounter++;
            insertStartCommand = (pageCounter == startPageIndex);
            insertEndCommand = (pageCounter == endPageIndex + 1);
        }

        if (insertStartCommand && line.match(/%%BeginPageSetup/)) {
            newContent.push(line);
            newContent.push(startCommand);
            insertStartCommand = false;
        } else if (insertEndCommand && line.match(/%%Page:/)) {
            newContent.push(endCommand);
            newContent.push(line);
            insertEndCommand = false;
        } else {
            newContent.push(line);
        }
    }

    return newContent.join("\n");
}

// Main function
function main(filePath, startPageIndex, endPageIndex, startCommandLines, endCommandLines) {
    var file = openPSFile(filePath);
    if (file === null) return;

    var content = readPSFile(file);
    file.Close();

    var startCommand = startCommandLines.join("\n");
    var endCommand = endCommandLines.join("\n");

    var newContent = insertCommands(content, startPageIndex, endPageIndex, startCommand, endCommand);

    saveAndCloseFile(filePath, newContent);
}

// Example usage
var filePath = "path\\to\\your\\file.ps";
var startPageIndex = 1; // Start binding from this page index
var endPageIndex = 3; // End binding at this page index
var startCommandLines = [
    "%%BeginFeature: *FFStaple UpperLeftSinglePort",
    "3 dict begin",
    "    /Staple 3 def",
    "    /OutputType (FINISHER TRAY) def",
    "    2 dict begin",
    "    /Type 7 def",
    "    /Location 0 def currentdict end",
    "    /StapleDetails exch def currentdict end",
    "setpagedevice",
    "%%EndFeature",
    "} stopped cleartomark"
];

var endCommandLines = [
    "%%BeginFeature: *FFStaple None",
    "1 dict dup/Staple 0 put setpagedevice",
    "%%EndFeature",
    "} stopped cleartomark"
];

main(filePath, startPageIndex, endPageIndex, startCommandLines, endCommandLines);
